<a href="https://colab.research.google.com/github/arpitpatelsitapur/my-py-torch-journey/blob/main/PytorchANN_Optuna_MLFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision keras torchinfo optuna mlflow pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
!ngrok config add-authtoken 2uozsZtTlNlVorgvP1ZHAQwxcWF_4eCAUaWxnpDjkRcvtq5vH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import subprocess
from pyngrok import ngrok

# Kill any process using port 5000
!kill -9 $(lsof -t -i:5000) 2>/dev/null

# Start MLflow UI
mlflow_proc = subprocess.Popen(["mlflow", "ui", "--port", "5000"])
ngrok_tunnel = ngrok.connect(5000)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://9d2ddb144cd8.ngrok-free.app


In [6]:
!pip install optuna-integration[mlflow]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 4.6 MB/s eta 0:00:00


In [7]:
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from keras import datasets
import mlflow
import mlflow.pytorch
from optuna.integration.mlflow import MLflowCallback

torch.manual_seed(7)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load Fashion-MNIST dataset
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)

# convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.n_samples = X.shape[0]

    def __len__(self):
        return self.n_samples

    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset = CustomDataset(X_train_t, y_train_t)
test_dataset = CustomDataset(X_test_t, y_test_t)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
class MyNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, neurons_per_layer, dropout_rate):
        super(MyNN, self).__init__()
        layers = []
        for i in range(hidden_size):
            layers.append(nn.Linear(input_size, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_size = neurons_per_layer
        layers.append(nn.Linear(neurons_per_layer, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [9]:
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 2, 24)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 256, step=8)
    input_size, output_size = 784, 10
    num_epochs = trial.suggest_int("num_epochs", 10, 50, step=10)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5, step=0.1)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])

    # dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # model
    model = MyNN(input_size, hidden_size, output_size, neurons_per_layer, dropout_rate).to(device)

    # optimizer
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

    loss_fn = nn.CrossEntropyLoss()

    # training loop
    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            l = loss_fn(outputs, batch_y.squeeze(1))
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

    # evaluation
    model.eval()
    n_correct, n_samples = 0, 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += batch_y.size(0)
            n_correct += (predicted == batch_y.squeeze(1)).sum().item()

    acc = n_correct / n_samples

    # log model artifact for each trial
    with mlflow.start_run(nested=True):
        mlflow.pytorch.log_model(model, f"model_trial_{trial.number}")

    return acc

In [10]:
mlflow.set_experiment("pytorch_fashion_mnist_optuna")
mlflc = MLflowCallback(metric_name="accuracy")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, callbacks=[mlflc])  # use more trials if you want

print("Best accuracy:", study.best_trial.value)
print("Best params:", study.best_trial.params)

2025/09/20 10:11:19 INFO mlflow.tracking.fluent: Experiment with name 'pytorch_fashion_mnist_optuna' does not exist. Creating a new experiment.
/tmp/ipython-input-3817243680.py:2: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(metric_name="accuracy")
[I 2025-09-20 10:11:19,265] A new study created in memory with name: no-name-a4840b2c-afc3-4ff8-b07d-d5d3b0cca43d
2025/09/20 10:20:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 10:20:09 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0+cu126) contains a local version label (+cu126). MLflow logged a pip requirement for this package as 'torch==2.8.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/09/20 10:20:20 WARNING mlflow.utils.requirements_uti

KeyboardInterrupt: 